##     Analizador para el dataset de detección de vehículos en fotos aéreas.

##  Determine que significa la salida de los txt 

Los datos siguen el formato de anotación YOLO (You Only Look Once). Cada línea representa un objeto detectado y los valores son coordenadas normalizadas (entre 0 y 1) relativas a las dimensiones totales de la imagen.

Para el ejemplo: 0 0.167447... 0.496759... 0.036979... 0.034259...

0 (Clase): Es el ID de la clase del objeto. En este dataset específico, el 0 corresponde a Car.  

Uno del os archivos contenía la descripción de las clases:
            0: "car",
            1: "truck",
            2: "bus",
            3: "minibus",
            4: "cyclist"

0.1674... (Centro X): Es la posición horizontal del centro del objeto. Significa que el centro del coche está al 16.74% del ancho de la imagen (empezando desde la izquierda).

0.4967... (Centro Y): Es la posición vertical del centro del objeto. El centro del coche está al 49.67% de la altura de la imagen (empezando desde arriba).

0.0369... (Ancho): Es el ancho del objeto. El coche ocupa el 3.69% del ancho total de la imagen.

0.0342... (Alto): Es la altura del objeto. El coche ocupa el 3.42% de la altura total de la imagen.

## Algoritmo en Python que devide en Train y Test

In [1]:
import os
import shutil
from pathlib import Path
import random
from collections import defaultdict
import cv2
import numpy as np

class AerialCarsAnalyzer:
    """
    Analizador para el dataset de detección de vehículos en fotos aéreas.
    Soporta 5 clases y ajusta índices (1-5 -> 0-4) para YOLO.
    """
    
    def __init__(self, dataset_path):
        self.dataset_path = Path(dataset_path)
        self.images_path = self.dataset_path / "images"
        self.labels_path = self.dataset_path / "labels"
        
        # Definimos las 5 clases correctamente (formato YOLO: 0 a 4)
        self.class_names = {
            0: "car",
            1: "truck",
            2: "bus",
            3: "minibus",
            4: "cyclist"
        }
        
    def parse_yolo_annotation(self, txt_file):
        """
        Lee y parsea un archivo de anotación.
        IMPORTANTE: Convierte clases 1-5 (del dataset original) a 0-4 (para YOLO).
        """
        annotations = []
        
        if not os.path.exists(txt_file):
            return annotations
            
        with open(txt_file, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    raw_class_id = int(parts[0])
                    
                    # Lógica de corrección de índice
                    # Si el dataset viene como 1=car, 2=truck... restamos 1.
                    # Si viene 0=car, lo dejamos igual.
                    if raw_class_id > 0:
                        class_id = raw_class_id - 1
                    else:
                        class_id = raw_class_id

                    x_center = float(parts[1])
                    y_center = float(parts[2])
                    width = float(parts[3])
                    height = float(parts[4])
                    
                    # Solo agregamos si la clase es válida (0 a 4)
                    if class_id in self.class_names:
                        annotations.append({
                            'class_id': class_id,
                            'class_name': self.class_names[class_id],
                            'x_center': x_center,
                            'y_center': y_center,
                            'width': width,
                            'height': height
                        })
        
        return annotations
    
    def analyze_dataset(self):
        """
        Analiza todo el dataset y genera estadísticas
        """
        print("=" * 60)
        print("ANÁLISIS DEL DATASET AERIAL CARS (MULTICLASE)")
        print("=" * 60)
        
        total_images = 0
        total_objects = 0
        class_counts = defaultdict(int)
        objects_per_image = []
        
        label_files = list(self.labels_path.glob("*.txt"))
        
        if len(label_files) == 0:
            print(f" ERROR: No se encontraron archivos .txt en {self.labels_path}")
            return {}

        print(f"\n Archivos de etiquetas encontrados: {len(label_files)}")
        
        for label_file in label_files:
            # Buscar imagen asociada
            image_name = label_file.stem
            image_extensions = ['.jpg', '.jpeg', '.png', '.JPG']
            image_found = False
            
            for ext in image_extensions:
                if (self.images_path / f"{image_name}{ext}").exists():
                    image_found = True
                    break
            
            if not image_found:
                continue
                
            total_images += 1
            annotations = self.parse_yolo_annotation(label_file)
            
            objects_count = len(annotations)
            objects_per_image.append(objects_count)
            total_objects += objects_count
            
            for ann in annotations:
                class_counts[ann['class_name']] += 1
        
        # Mostrar estadísticas
        print(f"\n Total de imágenes válidas: {total_images}")
        print(f" Total de objetos detectados: {total_objects}")
        
        print(f"\n DISTRIBUCIÓN POR CLASES:")
        print("-" * 60)
        for i in range(5): # Iteramos en orden para ver todas, incluso las vacías
            name = self.class_names[i]
            count = class_counts[name]
            percentage = (count / total_objects * 100) if total_objects > 0 else 0
            print(f"  ID {i} [{name}]: {count} objetos ({percentage:.2f}%)")
        
        return {'total_images': total_images}
    
    def split_dataset(self, train_ratio=0.8, output_dir="dataset_split", seed=42):
        """
        Divide el dataset en train y test y corrige las etiquetas en los archivos nuevos.
        """
        random.seed(seed)
        output_path = Path(output_dir)
        
        # Limpiar directorio previo si existe para evitar mezclas
        if output_path.exists():
            shutil.rmtree(output_path)
        
        # Crear directorios
        train_images_dir = output_path / "train" / "images"
        train_labels_dir = output_path / "train" / "labels"
        test_images_dir = output_path / "test" / "images"
        test_labels_dir = output_path / "test" / "labels"
        
        for p in [train_images_dir, train_labels_dir, test_images_dir, test_labels_dir]:
            p.mkdir(parents=True, exist_ok=True)
            
        label_files = list(self.labels_path.glob("*.txt"))
        random.shuffle(label_files)
        
        split_idx = int(len(label_files) * train_ratio)
        train_files = label_files[:split_idx]
        test_files = label_files[split_idx:]
        
        def process_files(files, img_dest, lbl_dest):
            for label_file in files:
                # 1. Leer y corregir etiquetas
                annotations = self.parse_yolo_annotation(label_file)
                
                # 2. Guardar nuevo archivo de etiquetas corregido
                with open(lbl_dest / label_file.name, 'w') as f:
                    for ann in annotations:
                        # Escribimos el class_id ya corregido (0-4)
                        line = f"{ann['class_id']} {ann['x_center']} {ann['y_center']} {ann['width']} {ann['height']}\n"
                        f.write(line)
                
                # 3. Copiar imagen
                image_name = label_file.stem
                for ext in ['.jpg', '.jpeg', '.png', '.JPG']:
                    src_img = self.images_path / f"{image_name}{ext}"
                    if src_img.exists():
                        shutil.copy2(src_img, img_dest / src_img.name)
                        break

        print("\n Procesando conjunto de ENTRENAMIENTO...")
        process_files(train_files, train_images_dir, train_labels_dir)
        
        print(" Procesando conjunto de PRUEBA (VALIDACIÓN)...")
        process_files(test_files, test_images_dir, test_labels_dir)
        
        # Generar dataset.yaml con las 5 clases
        config_content = f"""path: {output_path.absolute()}
train: train/images
val: test/images

# Número de clases
nc: 5

# Nombres de clases
names: ['car', 'truck', 'bus', 'minibus', 'cyclist']
"""
        with open(output_path / "dataset.yaml", 'w') as f:
            f.write(config_content)
            
        print(f"\n Dataset generado correctamente en: {output_dir}")
        print(f"📄 Archivo de configuración: {output_path / 'dataset.yaml'}")

# --- Ejecución ---
if __name__ == "__main__":
    
    dataset_path = "aerial-cars-dataset-master" 
    
    if os.path.exists(dataset_path):
        analyzer = AerialCarsAnalyzer(dataset_path)
        analyzer.analyze_dataset()
        analyzer.split_dataset()
    else:
        print(f"No encuentro la carpeta {dataset_path}")

ANÁLISIS DEL DATASET AERIAL CARS (MULTICLASE)

 Archivos de etiquetas encontrados: 165

 Total de imágenes válidas: 154
 Total de objetos detectados: 4025

 DISTRIBUCIÓN POR CLASES:
------------------------------------------------------------
  ID 0 [car]: 3800 objetos (94.41%)
  ID 1 [truck]: 104 objetos (2.58%)
  ID 2 [bus]: 121 objetos (3.01%)
  ID 3 [minibus]: 0 objetos (0.00%)
  ID 4 [cyclist]: 0 objetos (0.00%)

 Procesando conjunto de ENTRENAMIENTO...
 Procesando conjunto de PRUEBA (VALIDACIÓN)...

 Dataset generado correctamente en: dataset_split
📄 Archivo de configuración: dataset_split\dataset.yaml
